# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka)** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Install Kafka SERVER

Go to `kafka` folder and run:

`docker compose up -d`

# Create SparkSession
## Install Kafka connector

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-dc337b0a-325d-48be-b11c-6cd79141c571;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

# Create a data stream from a Kafka topic

In [2]:
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-example-1") \
            .load()

kafka_df.printSchema()

# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")

# Send transformed data to the Sink
query_a = df_input.writeStream \
            .trigger(processingTime='1 second') \
            .outputMode("append") \
            .format("console") \
            .start()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



25/10/10 00:55:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-85f88ff5-b45a-4727-b77a-4452a4f20930. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/10 00:55:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+
|value|
+-----+
+-----+



25/10/10 00:55:35 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 1224 milliseconds
25/10/10 00:57:11 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 1000} milliseconds, but spent 1526 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
|hello|
+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+
| value|
+------+
|hello2|
+------+



In [3]:
query_a.stop()
sc.stop()

25/10/10 01:00:24 WARN DAGScheduler: Failed to cancel job group 2c43751c-c424-4286-a2f4-cf3d4311e927. Cannot find active jobs for it.
25/10/10 01:00:24 WARN DAGScheduler: Failed to cancel job group 2c43751c-c424-4286-a2f4-cf3d4311e927. Cannot find active jobs for it.


## Add a Streaming listener

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pcamarillor.streaming_listener import MyListener

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Add custom listener
spark.streams.addListener(MyListener())

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2.5.2/cache
The jars for the packages stored in: /root/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f4171cb9-3e32-4d8f-921f-44b06b6fb8e7;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.0 in central
	found org.apache.kafka#kafka-clients;3.9.0 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.7 in central
	found org.slf4j#slf4j-api;2.0.16 in central
	found org.apache.hadoop#hadoop-client-runtime;3.4.1 in central
	found org.apache.hadoop#hadoop-client-api;3.4.1 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.2.0

Query started: 17fd8bdf-8b6e-49ca-8cc2-faec0baa94fb
Query made progress: {
  "id" : "17fd8bdf-8b6e-49ca-8cc2-faec0baa94fb",
  "runId" : "5cac179c-d1e1-4121-abdb-10a1be1c2507",
  "name" : null,
  "timestamp" : "2025-10-10T01:19:30.281Z",
  "batchId" : 0,
  "batchDuration" : 2263,
  "numInputRows" : 0,
  "inputRowsPerSecond" : 0.0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "addBatch" : 1630,
    "commitOffsets" : 85,
    "getBatch" : 13,
    "latestOffset" : 243,
    "queryPlanning" : 268,
    "triggerExecution" : 2262,
    "walCommit" : 16
  },
  "stateOperators" : [ {
    "operatorName" : "stateStoreSave",
    "numRowsTotal" : 0,
    "numRowsUpdated" : 0,
    "allUpdatesTimeMs" : 38,
    "numRowsRemoved" : 0,
    "allRemovalsTimeMs" : 0,
    "commitTimeMs" : 85,
    "memoryUsedBytes" : 1200,
    "numRowsDroppedByWatermark" : 0,
    "numShufflePartitions" : 5,
    "numStateStoreInstances" : 5,
    "customMetrics" : {
      "loadedMapCacheHitCount" : 0,
      "loadedMapCac

In [4]:
from pyspark.sql.functions import explode, split
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-example-1") \
            .load()


# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")
df_input = df_input.select(explode(split(df_input.value, " ")).alias("word"))
df_input = df_input.groupBy("word").count()

# Send transformed data to the Sink
query_b = df_input.writeStream \
            .trigger(processingTime='3 second') \
            .outputMode("complete") \
            .format("console") \
            .start()


query_b.awaitTermination(300)

25/10/10 01:21:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-15ba6b8f-c881-4921-b0c2-93f6325330de. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/10 01:21:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|   hi|    1|
| name|    1|
|pablo|    1|
|   my|    1|
|   is|    1|
+-----+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|   hi|    1|
| name|    1|
|   my|    1|
|   is|    1|
|pablo|    1|
|again|    1|
|hello|    1|
+-----+-----+



False

In [ ]:
query_b.stop()
sc.stop()